## Preprocessing EPL Data from 2013-2019 for Analytics Application

In [2]:
import numpy as np
import pandas as pd

In [12]:
_1314data = pd.read_csv('./data/13-14.csv')
_1415data = pd.read_csv('./data/14-15.csv')
_1516data = pd.read_csv('./data/15-16.csv')
_1617data = pd.read_csv('./data/16-17.csv')
_1718data = pd.read_csv('./data/17-18.csv')
_1819data = pd.read_csv('./data/18-19.csv')
seasons_data = [_1314data, _1415data, _1516data, _1617data, _1718data, _1819data]

### Preliminary Tests
Before combining, we run some preliminary tests to determine the validity of the data. This includes:
1. Ensuring that each season's dataset has 380 games. There are 20 teams and they play each other twice. There are thus $2 \cdot {20 \choose 2} = 380$ games.
2. Ensuring that each team has played 38 games a season.
3. Ensuring that each team has played an equal number (19) of home and away games.

In [13]:
def test_number_of_games_in_season(df):
    """
    Determines the number of games in a season's dataset.
    """
    if df.shape[0] == 380:
        print('Seasons data has the right number of games')
    elif df.shape[0] < 380:
        print('Seasons data is missing some games')
    else:
        print('Seasons data has more the necessary games')

In [14]:
for df in seasons_data:
    test_number_of_games_in_season(df)    

Seasons data has the right number of games
Seasons data has more the necessary games
Seasons data has the right number of games
Seasons data has the right number of games
Seasons data has the right number of games
Seasons data has the right number of games


In [15]:
#2014-15 season has an extra all-NAN row.
_1415data.dropna(how='all', inplace=True)

In [16]:
for df in seasons_data:
    test_number_of_games_in_season(df)    

Seasons data has the right number of games
Seasons data has the right number of games
Seasons data has the right number of games
Seasons data has the right number of games
Seasons data has the right number of games
Seasons data has the right number of games


In [17]:
def check_home_team_games_played(df):
    home_team_games = df['HomeTeam'].value_counts()
    if len(home_team_games) == 20:
        print('All teams have played home games')
    for game in home_team_games:
        if game < 19:
            print('Team has not played enough games')
        if game > 19:
            print('Team has played more than enough games')

In [18]:
for df in seasons_data:
    check_home_team_games_played(df)

All teams have played home games
All teams have played home games
All teams have played home games
All teams have played home games
All teams have played home games
All teams have played home games


In [19]:
def check_away_team_games_played(df):
    away_team_games = df['AwayTeam'].value_counts()
    if len(away_team_games) == 20:
        print('All teams have played away games')
    for game in away_team_games:
        if game < 19:
            print('Team has not played enough games')
        if game > 19:
            print('Team has played more than enough games')

In [20]:
for df in seasons_data:
    check_away_team_games_played(df)

All teams have played away games
All teams have played away games
All teams have played away games
All teams have played away games
All teams have played away games
All teams have played away games


### Column Name Tests

The data was obtained on a season-by-season basis and sometimes came from a different source. Naturally, there might be the following discrepancies:
1. Different column names that refer to the same property (e.g. "HomeTeam" vs "hometeam")
2. Columns in one season's dataset not present in another season's dataset.
3. Columns that have some data missing.

In [21]:
for df in seasons_data:
    print(df.columns)

Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC',
       'AC', 'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD',
       'BWA', 'IWH', 'IWD', 'IWA', 'LBH', 'LBD', 'LBA', 'PSH', 'PSD', 'PSA',
       'WHH', 'WHD', 'WHA', 'SJH', 'SJD', 'SJA', 'VCH', 'VCD', 'VCA', 'Bb1X2',
       'BbMxH', 'BbAvH', 'BbMxD', 'BbAvD', 'BbMxA', 'BbAvA', 'BbOU',
       'BbMx>2.5', 'BbAv>2.5', 'BbMx<2.5', 'BbAv<2.5', 'BbAH', 'BbAHh',
       'BbMxAHH', 'BbAvAHH', 'BbMxAHA', 'BbAvAHA'],
      dtype='object')
Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC',
       'AC', 'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD',
       'BWA', 'IWH', 'IWD', 'IWA', 'LBH', 'LBD', 'LBA', 'PSH', 'PSD', 'PSA',
       'WHH', 'WHD', 'WHA', 'SJH', 'SJD', 'SJA', 'VCH', 'VCD', 'VCA', 'Bb1X2',
       'B

### Initial Observations
- 2013-14 and 2014-15 data have some additional data and also have some different column names for some of the same properties.
- The data we have that is common between all the season's data are the following:
    1. Date 
    2. HomeTeam
    3. AwayTeam
    4. FTHG
    5. FTAG
    6. FTR
    7. HTHG
    8. HTAG
    9. HTR
    10. Referee
    11. HS
    12. AS
    13. HST
    14. AST
    15. HF
    16. AF
    17. HC
    18. AC
    19. HY
    20. AY
    21. HR
    22. AR
    
- Given that we cannot perform analysis on sparse data, we will be exluding the additional data present in the 2013-14 and 2014-15 dataset.

In [22]:
columns = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC',
       'AC', 'HY', 'AY', 'HR', 'AR']

In [23]:
# Obtaining the necessary subset of data for each season
for i in range(len(seasons_data)):
    df = seasons_data[i]
    seasons_data[i] = df[columns]

In [24]:
for df in seasons_data:
    print(df.columns)

Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG',
       'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC',
       'HY', 'AY', 'HR', 'AR'],
      dtype='object')
Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG',
       'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC',
       'HY', 'AY', 'HR', 'AR'],
      dtype='object')
Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG',
       'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC',
       'HY', 'AY', 'HR', 'AR'],
      dtype='object')
Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG',
       'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC',
       'HY', 'AY', 'HR', 'AR'],
      dtype='object')
Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG',
       'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC',
       'HY', 'AY', 

### Checking for null values
This is just a routine check to ensure we don't push any bad data into the databases.

In [25]:
for df in seasons_data:
    print(df.isnull().sum())

Date        0
HomeTeam    0
AwayTeam    0
FTHG        0
FTAG        0
FTR         0
HTHG        0
HTAG        0
HTR         0
Referee     0
HS          0
AS          0
HST         0
AST         0
HF          0
AF          0
HC          0
AC          0
HY          0
AY          0
HR          0
AR          0
dtype: int64
Date        0
HomeTeam    0
AwayTeam    0
FTHG        0
FTAG        0
FTR         0
HTHG        0
HTAG        0
HTR         0
Referee     0
HS          0
AS          0
HST         0
AST         0
HF          0
AF          0
HC          0
AC          0
HY          0
AY          0
HR          0
AR          0
dtype: int64
Date        0
HomeTeam    0
AwayTeam    0
FTHG        0
FTAG        0
FTR         0
HTHG        0
HTAG        0
HTR         0
Referee     0
HS          0
AS          0
HST         0
AST         0
HF          0
AF          0
HC          0
AC          0
HY          0
AY          0
HR          0
AR          0
dtype: int64
Date        0
HomeTeam    0
AwayTeam 

### Joining the Seasons data to form the final dataset
- We have the necessary columns for the application. Joining it is a simple matter of joining dataframes.
- Before joining the dataframes, we add an additional column to each of them named 'Season' and add the relevant years for the season. For example, any games played during the 2013-14 season will have a value of '2013-14' under the 'Season' tag.

In [26]:
seasons = ['2013-14', '2014-15', '2015-16', '2016-17', '2017-18', '2018-19']
SEASON_COLUMN_NAME = 'Season'

In [27]:
for i in range(len(seasons_data)):
    df = seasons_data[i]
    df[SEASON_COLUMN_NAME] = seasons[i]
    seasons_data[i] = df

/Users/vidhur2k/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [28]:
for df in seasons_data:
    assert df[SEASON_COLUMN_NAME] is not None

In [29]:
dataset = pd.DataFrame([])

In [30]:
for df in seasons_data:
    dataset = pd.concat([dataset, df], ignore_index=True)

In [31]:
dataset

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,AST,HF,AF,HC,AC,HY,AY,HR,AR,Season
0,17/08/13,Arsenal,Aston Villa,1.0,3.0,A,1.0,1.0,D,A Taylor,...,4.0,15.0,18.0,4.0,3.0,4.0,5.0,1.0,0.0,2013-14
1,17/08/13,Liverpool,Stoke,1.0,0.0,H,1.0,0.0,H,M Atkinson,...,4.0,11.0,11.0,12.0,6.0,1.0,1.0,0.0,0.0,2013-14
2,17/08/13,Norwich,Everton,2.0,2.0,D,0.0,0.0,D,M Oliver,...,6.0,13.0,10.0,6.0,8.0,2.0,0.0,0.0,0.0,2013-14
3,17/08/13,Sunderland,Fulham,0.0,1.0,A,0.0,0.0,D,N Swarbrick,...,1.0,14.0,14.0,6.0,1.0,0.0,3.0,0.0,0.0,2013-14
4,17/08/13,Swansea,Man United,1.0,4.0,A,0.0,2.0,A,P Dowd,...,7.0,13.0,10.0,7.0,4.0,1.0,3.0,0.0,0.0,2013-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2275,12/5/19,Liverpool,Wolves,2.0,0.0,H,1.0,0.0,H,M Atkinson,...,2.0,3.0,11.0,4.0,1.0,0.0,2.0,0.0,0.0,2018-19
2276,12/5/19,Man United,Cardiff,0.0,2.0,A,0.0,1.0,A,J Moss,...,4.0,9.0,6.0,11.0,2.0,3.0,3.0,0.0,0.0,2018-19
2277,12/5/19,Southampton,Huddersfield,1.0,1.0,D,1.0,0.0,H,L Probert,...,3.0,8.0,6.0,4.0,3.0,0.0,1.0,0.0,0.0,2018-19
2278,12/5/19,Tottenham,Everton,2.0,2.0,D,1.0,0.0,H,A Marriner,...,9.0,10.0,13.0,7.0,4.0,0.0,2.0,0.0,0.0,2018-19


### Final Tests
Now that we have the final dataset, we just need to do some final checks before exporting to CSV:
1. Ensure that there are $380x6 = 2280$ (6 seasons) rows in the data.
2. Ensure that the number of columns are the same as the number of columns common to all the seasons data.
3. Ensure that we do not have any null values

In [32]:
def perform_final_dataset_assertions(df):
    assert df.shape[0] == 2280
    assert df.shape[1] == len(columns) + 1 # + 1 for the 'Seasons' column
    assert sum(df.isnull().sum()) == 0

In [33]:
perform_final_dataset_assertions(dataset)

### Different CSVs for Each Entity

In [24]:
# Referee
referees_df = dataset[['Referee']]

In [30]:
referees_df = referees_df.drop_duplicates()
referees_df

,Referee
0,A Taylor
1,M Atkinson
2,M Oliver
3,N Swarbrick
4,P Dowd
5,K Friend
6,H Webb
7,J Moss
8,M Clattenburg
9,A Marriner


In [28]:
referees_df.to_csv('./data/referees.csv', index=False)

In [31]:
# Team
teams_df = dataset[['HomeTeam']]
teams_df = teams_df.drop_duplicates()

In [90]:
list(teams_df['HomeTeam'])

['Arsenal',
 'Liverpool',
 'Norwich',
 'Sunderland',
 'Swansea',
 'West Brom',
 'West Ham',
 'Chelsea',
 'Crystal Palace',
 'Man City',
 'Aston Villa',
 'Everton',
 'Fulham',
 'Hull',
 'Newcastle',
 'Southampton',
 'Stoke',
 'Cardiff',
 'Tottenham',
 'Man United',
 'Leicester',
 'QPR',
 'Burnley',
 'Bournemouth',
 'Watford',
 'Middlesbrough',
 'Brighton',
 'Huddersfield',
 'Wolves']

In [35]:
teams_df.to_csv('./data/teams.csv', index=False)

## Making the Game Entity

In [44]:
def parse_id(df):
    df['_id'] = df['_id'].apply(lambda x: x[9:-1])
    return df

In [9]:
def make_dict(df):
    out = {}
    for _, row in df.iterrows():
        out[row['Name']] = row['_id']
    return out
def make_postgres_dict(df):
    out = {}
    for _, row in df.iterrows():
        out[row['name']] = row['id']
    return out

In [82]:
m_referees = pd.read_csv('./data/mongo/referees.csv')
parse_id(m_referees)
referees_dict = make_dict(m_referees)
referees_dict.items()

dict_items([('Atkinson', '5fc451059280f04a4cad63c7'), ('Taylor', '5fc451059280f04a4cad63c8'), ('Swarbrick', '5fc451059280f04a4cad63c9'), ('Jones', '5fc451059280f04a4cad63ca'), ('Webb', '5fc451059280f04a4cad63cb'), ('Oliver', '5fc451059280f04a4cad63cc'), ('Probert', '5fc451059280f04a4cad63cd'), ('Mason', '5fc451059280f04a4cad63dd'), ('Friend', '5fc451059280f04a4cad63cf'), ('Moss', '5fc451059280f04a4cad63d0'), ('Pawson', '5fc451059280f04a4cad63d1'), ('Madley', '5fc451059280f04a4cad63e1'), ('East', '5fc451059280f04a4cad63d3'), ('Clattenburg', '5fc451059280f04a4cad63d4'), ('Dowd', '5fc451059280f04a4cad63d5'), ('Tierney', '5fc451059280f04a4cad63d6'), ('Foy', '5fc451059280f04a4cad63d7'), ('Marriner', '5fc451059280f04a4cad63d8'), ('Stroud', '5fc451059280f04a4cad63d9'), ('Dean', '5fc451059280f04a4cad63da'), ('Hooper', '5fc451059280f04a4cad63db'), ('Attwell', '5fc451059280f04a4cad63dc'), ('Scott', '5fc451059280f04a4cad63de'), ('Kavanagh', '5fc451059280f04a4cad63df'), ('Coote', '5fc451059280f04a

In [83]:
m_teams = pd.read_csv('./data/mongo/teams.csv')
parse_id(m_teams)
teams_dict = make_dict(m_teams)
teams_dict.items()

dict_items([('West Brom', '5fc4529d9280f04a4cadda35'), ('Everton', '5fc4529d9280f04a4cadda36'), ('Fulham', '5fc4529d9280f04a4cadda37'), ('West Ham', '5fc4529d9280f04a4cadda38'), ('Hull', '5fc4529d9280f04a4cadda39'), ('Newcastle', '5fc4529d9280f04a4cadda3a'), ('Southampton', '5fc4529d9280f04a4cadda3b'), ('Stoke', '5fc4529d9280f04a4cadda3c'), ('Arsenal', '5fc4529d9280f04a4cadda3d'), ('Cardiff', '5fc4529d9280f04a4cadda3e'), ('Man City', '5fc4529d9280f04a4cadda3f'), ('Norwich', '5fc4529d9280f04a4cadda40'), ('Man United', '5fc4529d9280f04a4cadda41'), ('Leicester', '5fc4529d9280f04a4cadda42'), ('Swansea', '5fc4529d9280f04a4cadda43'), ('QPR', '5fc4529d9280f04a4cadda44'), ('Burnley', '5fc4529d9280f04a4cadda45'), ('Chelsea', '5fc4529d9280f04a4cadda46'), ('Crystal Palace', '5fc4529d9280f04a4cadda47'), ('Watford', '5fc4529d9280f04a4cadda48'), ('Tottenham', '5fc4529d9280f04a4cadda49'), ('Bournemouth', '5fc4529d9280f04a4cadda4a'), ('Sunderland', '5fc4529d9280f04a4cadda4b'), ('Aston Villa', '5fc4529

In [84]:
dataset2 = dataset.copy()

In [85]:
dataset2['HomeTeam'] = dataset2['HomeTeam'].apply(lambda team: teams_dict[team])

In [86]:
dataset2['AwayTeam'] = dataset2['AwayTeam'].apply(lambda team: teams_dict[team])

In [87]:
# Ignoring the referees' initial
dataset2['Referee'] = dataset2['Referee'].apply(lambda referee: referees_dict[referee.split()[1]])

In [88]:
dataset2.to_csv('./data/games.csv', index=False)

## Make the Game Entity (PostgreSQL)

In [10]:
m_referees = pd.read_csv('./data/postgres/referees.csv', names=['id', 'name'])
referees_dict = make_postgres_dict(m_referees)
referees_dict.items()

dict_items([('Taylor', 1), ('Atkinson', 2), ('Oliver', 3), ('Swarbrick', 4), ('Dowd', 5), ('Friend', 6), ('Webb', 7), ('Moss', 8), ('Clattenburg', 9), ('Marriner', 10), ('East', 11), ('Jones', 12), ('Mason', 25), ('Probert', 14), ('Foy', 15), ('Dean', 16), ('Madley', 27), ('Pawson', 18), ('Tierney', 19), ('Stroud', 20), ('Attwell', 21), ('Scott', 22), ('Hooper', 23), ('Kavanagh', 24), ('Coote', 26)])

In [11]:
m_teams = pd.read_csv('./data/postgres/teams.csv', names=['id', 'name'])
teams_dict = make_postgres_dict(m_teams)
teams_dict.items()

dict_items([('Arsenal', 1), ('Liverpool', 2), ('Norwich', 3), ('Sunderland', 4), ('Swansea', 5), ('West Brom', 6), ('West Ham', 7), ('Chelsea', 8), ('Crystal Palace', 9), ('Man City', 10), ('Aston Villa', 11), ('Everton', 12), ('Fulham', 13), ('Hull', 14), ('Newcastle', 15), ('Southampton', 16), ('Stoke', 17), ('Cardiff', 18), ('Tottenham', 19), ('Man United', 20), ('Leicester', 21), ('QPR', 22), ('Burnley', 23), ('Bournemouth', 24), ('Watford', 25), ('Middlesbrough', 26), ('Brighton', 27), ('Huddersfield', 28), ('Wolves', 29)])

In [34]:
dataset3 = dataset.copy()

In [36]:
dataset3['HomeTeam'] = dataset3['HomeTeam'].apply(lambda team: teams_dict[team])

In [37]:
dataset3['AwayTeam'] = dataset3['AwayTeam'].apply(lambda team: teams_dict[team])

In [39]:
dataset3['Referee'] = dataset3['Referee'].apply(lambda referee: referees_dict[referee.split()[1]])

In [40]:
dataset3.to_csv('./data/postgres/games.csv', index=False)

### Exporting to CSV

In [ ]:
dataset.to_csv('./data/final_data.csv', index=False)

In [ ]:
df = pd.read_csv('./data/final_data.csv')

In [ ]:
# Ensuring consistency of the exported CSV when imported again
perform_final_dataset_assertions(df)